In [163]:
import numpy as np
import cv2
from cv2 import aruco
import pandas as pd
import matplotlib.colors as mcolors
# import random

In [164]:
dictionary = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)

In [165]:
def imshow(img, window_name="img"):
    cv2.startWindowThread()
    cv2.imshow(window_name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [166]:
# 鮮鋭化フィルタ
def applyFiltersSp(img, k=2):
    sharp_kernel = np.array([
        [-k / 9, -k / 9, -k / 9],
        [-k / 9, 1 + 8 * k / 9, k / 9],
        [-k / 9, -k / 9, -k / 9]
    ], np.float32)
    img_sp = cv2.filter2D(img, -1, sharp_kernel).astype("uint8")
    return img_sp

# バイラテラルフィルタ
def applyFiltersBltrl(img, d=15):
    img_bltrl = cv2.bilateralFilter(src=img, d=d, sigmaColor=75, sigmaSpace=75)
    return img_bltrl

In [167]:
# 軌跡データ
# csv_path = "output/test/markers_t_hd_60.csv"  # フィルタ適用前
# csv_path = "output/test/markers_t_h_60_comlement_filter_bltrl_hybrid_2.csv" # フィルタ適用後+スプライン補完
# csv_path = "output/test/markers_t_h_60_liner_comlement_filter_bltrl_hybrid.csv" # フィルタ適用後+線形補完
csv_path = "output/test/markers_base_4k_60_liner_nofilter.csv"
movie_path = "src/mov/VibrationTest/base_4k_60.MOV"
markers_raw = pd.read_csv(csv_path, header=[0, 1])
frames = len(markers_raw)

# 解析で使用するidのデータだけ抜き出し
ids = list(map(lambda col: str(col), [12, 13, 14]))
markers = markers_raw[ids]

# 構造（線の接続関係）の定義
stractures = [[12,13,14]]

# 検出したい距離の定義
distancees = [[12,13],[13,14]]

# ストラクチャーに定義されているマーカーidのSeries
stractures_ids = pd.Series(sum(stractures, [])).drop_duplicates().sort_values()


# ids = pd.Series(markers.columns.levels[0])  # カラムのid列を取得
# ids_mask = pd.Series(map(lambda id: str(id).isnumeric(), ids))  # 数値を真、文字列を偽とするマスクを作成
# ids = ids.loc[ids_mask] # マスクを使って数値（id）のみを抜き出し

In [168]:

# 動画データ
cap = cv2.VideoCapture(movie_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))    # 動画のfps
ret, frame = cap.read()
h, w = frame.shape[:2]

# 動画サイズが大きすぎる場合はリサイズする
h_max = 800
if h > h_max:
    ratio = h / w
    w = h_max
    h = int(h_max * ratio)

# 保存設定
fmt = cv2.VideoWriter_fourcc('m', 'p', '4', 'v') 
writer = cv2.VideoWriter('./output/tracking_test.mp4', fmt, fps, (w, h))

cpxy = ['cpx', 'cpy']
pos = [None, None]
pos_van = [None, None]
pos_lnr = [None, None]
pos_spr = [None, None]
orbits_van = pd.DataFrame()  # 元データの軌跡

font = cv2.FONT_HERSHEY_SIMPLEX 

for id in ids:
    # print("id :", id)
    for i, cp in enumerate(cpxy):
        pos[i] = markers[str(id), cp]
        pos_van[i] = pos[i]
    orbits_van[id] = list(zip(pos_van[0], pos_van[1]))
orbits_van

,12,13,14
0,"(3596, 1264)","(262, 1329)","(1936, 1294)"
1,"(3596, 1264)","(262, 1329)","(1936, 1294)"
2,"(3597, 1265)","(263, 1331)","(1938, 1295)"
3,"(3600, 1269)","(264, 1334)","(1939, 1298)"
4,"(3601, 1270)","(265, 1335)","(1940, 1299)"
...,...,...,...
3635,"(3603, 1267)","(265, 1333)","(1941, 1297)"
3636,"(3605, 1266)","(263, 1334)","(1941, 1297)"
3637,"(3605, 1266)","(260, 1336)","(1941, 1300)"
3638,"(3605, 1266)","(268, 1334)","(1942, 1298)"


In [169]:
speed_x = pd.Series([], dtype=float)
speed_y = pd.Series([], dtype=float)
acceleration_x = pd.Series([], dtype=float)
acceleration_y = pd.Series([], dtype=float)

for f in range(frames):
    ret, img = cap.read()
    if not ret or cv2.waitKey(int(1000 / fps)) == 27:
        break
    img = cv2.resize(img, dsize=(w, h))

    cv2.putText(img, f"frame: {f}", (10,20), font, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
    # img = np.zeros((h, w, 3))

    # フィルター
    img_f = applyFiltersBltrl(img)
    img_f = applyFiltersSp(img_f)

    # マーカー認識
    corners, ids, rejectedImgPoints = aruco.detectMarkers(img_f, dictionary)
    # マーカー可視化
    aruco.drawDetectedMarkers(img, corners, ids, (0, 255, 255))

    # 定義した構造から線を描画（元データ利用）
    positions = np.array(list(orbits_van.iloc[f]))
    lines_from_stractures = []
    for line in stractures:
        for pos in line:
            lines_from_stractures.append(list(orbits_van.iloc[f][str(pos)]))
        # print(lines_from_stractures)
        cv2.polylines(img, np.array([lines_from_stractures]), False, (0,0,255), 2)
        lines_from_stractures = []

    # 座標間の距離を求めて描画
    for ids in distancees:
        pos1_x, pos1_y, pos1_z = [
            markers.iloc[f][(str(ids[0]), 'x')], 
            markers.iloc[f][(str(ids[0]), 'y')], 
            markers.iloc[f][(str(ids[0]), 'z')]
            ]
        pos2_x, pos2_y, pos2_z = [
            markers.iloc[f][(str(ids[1]), 'x')], 
            markers.iloc[f][(str(ids[1]), 'y')], 
            markers.iloc[f][(str(ids[1]), 'z')]
            ]
        distance = np.round(np.sqrt((pos1_x - pos2_x)**2 + (pos1_y - pos2_y)**2 + (pos1_z - pos2_z)**2), 2)
        write_pos = (int((markers.iloc[f][(str(ids[0]), 'cpx')] + markers.iloc[f][(str(ids[1]), 'cpx')])/2),
                     int((markers.iloc[f][(str(ids[0]), 'cpy')] + markers.iloc[f][(str(ids[1]), 'cpy')])/2))
        cv2.putText(img, f"[{ids[0]}]-[{ids[1]}]{distance}m", write_pos, font, 0.5, (0, 255, 255), 1, cv2.LINE_AA)

    # 座標から速度と加速度を求める
    if f > 1:
        for i, id in enumerate(stractures_ids):
            speed_x_current = round(((markers[(str(id), 'x')][f] - markers[(str(id), 'x')][f - 1])) / (fps / 1000), 2)
            if f < 3 : speed_x[i] = 0
            acceleration_x[i] = round((speed_x_current - speed_x[i]) / (fps / 1000), 2)
            speed_x[i] = speed_x_current
            txt = f"[{id}] spd x: {speed_x_current}m/sec | acc x: {acceleration_x[i]}m/sec^2"
            cv2.putText(img, txt, (10,40 + i * 20), font, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

    

    # # 通常データを描画（<NA>が入っていれば描画しない）
    # lines_pos_van = np.array(list(orbits_van.iloc[f]))
    # count_nan = pd.Series(list(map(lambda t: pd.Series(t).isna().sum() ,lines_pos_van))).sum()
    # # if count_nan == 0:
    # #     cv2.polylines(img, [lines_pos_van], True, (0,255,255), 2)

    
    cv2.imshow("mov", img)
    writer.write(img)
    # print(f, lines_pos_spr)

cv2.destroyAllWindows()
writer.release()